# Advanced Usage

Some more complex usages and tools

In [1]:
from jsonlitedb import JSONLiteDB
import sqlite3
import json

## Nested Dictionaries

One advantage of JSONLiteDB is that it can query *into* a nested dictionary. Consider the following: (obviosly, in real usages, there would be many more items)

In [2]:
db = JSONLiteDB(":memory:")
with db:  # Not needed but batches the insertion
    db.insert(
        {
            "first": "John",
            "last": "Smith",
            "phone": {"home": "215.555.6587", "work": "919.555.4795"},
            "kids": [
                {"first": "John Jr.", "last": "Smith"},
                {"first": "Jane", "last": "Smith"},
            ],
        }
    )
    db.insert(
        {
            "first": "Clark",
            "last": "Drake",
            "phone": {"home": "412.555.4960", "work": "410.555.9903"},
            "kids": [],
        }
    )
    db.insert(
        {
            "first": "Peggy",
            "last": "Line",
            "phone": {"home": "505.555.3101"},
            "kids": [
                {"first": "Jane", "last": "Line"},
                {"first": "Molly", "last": "Line"},
            ],
        }
    )
    db.insert(
        {
            "first": "Luke",
            "last": "Truss",
            "phone": {"home": "610.555.2647"},
            "kids": [{"first": "Janet", "last": "Truss"}],
        }
    )

All of these have **identical results**

In [3]:
# Using a tuple
list(db.query({("phone", "home"): "505.555.3101"}))

# Quoted JSON path. See below for discussion and comments on quoting
list(db.query({'$."phone"."home"': "505.555.3101"}))

# Query Objects
list(db.query(db.Q.phone.home == "505.555.3101"))

[{'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

This is also **identical** but since this is `"$.phone.home"` and the others result in `'$."phone"."home"'` an index built with one will not be used with the other.

In [4]:
# Using the SQLite JSON path style.
list(db.query({"$.phone.home": "505.555.3101"}))

[{'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

Of course, using a query object allows for even more flexibility: Use a LIKE

In [5]:
list(db.query(db.Q.phone.home % "505%"))

[{'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

### rowid

Every returned item is basically a dict with the addition of the `rowid` property

In [6]:
type(db.query_one(db.Q.phone.home % "505%"))

jsonlitedb.DBDict

In [7]:
db.query_one(db.Q.phone.home % "505%").rowid

3

And you can get items by rowid

In [8]:
db[3]

{'first': 'Peggy',
 'last': 'Line',
 'phone': {'home': '505.555.3101'},
 'kids': [{'first': 'Jane', 'last': 'Line'},
  {'first': 'Molly', 'last': 'Line'}]}

### Lists of Items

You can also query from a list element given the zero-based index.

Find any whos *first* kid is named 'Jane'. Both of the following are the same:

In [9]:
list(db.query({("kids", 0, "first"): "Jane"}))

[{'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

In [10]:
list(db.query(db.Q.kids[0].first == "Jane"))

[{'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

To query if *any* kid is named 'Jane', it is a bit harder. Actually, SQLite's [JSON_EACH](https://www.sqlite.org/json1.html#jeach) will do it but at the moment, this is not directly supported in JSONLiteDB. But you can make use JSONLiteDB to reduce the options then finish in Python. Note that this can still be a major speedup over pure Python.

In [11]:
items = []
qres = db.query(db.Q.kids % "%Jane%")
# Note, you don't want to end here since it will pick up 'Janet'
for item in qres:
    if any(kid["first"] == "Jane" for kid in item.get("kids", [])):
        items.append(item)
items

[{'first': 'John',
  'last': 'Smith',
  'phone': {'home': '215.555.6587', 'work': '919.555.4795'},
  'kids': [{'first': 'John Jr.', 'last': 'Smith'},
   {'first': 'Jane', 'last': 'Smith'}]},
 {'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

*Really* advanced usage is use SQL directly. This, of course, doesn't leverage JSONLiteDB at all...

In [12]:
qres = db.db.execute(
    """
    -- DISTINCT because JSON_EACH expands rows. Not needed in this case
    -- and can make repeated rows only show once. But works here.
    SELECT DISTINCT 
        items.data 
    FROM 
        items, 
        JSON_EACH(items.data,'$."kids"') AS kids
    WHERE
        JSON_EXTRACT(kids.value,'$."first"') = 'Jane'
    """
)
items = [json.loads(row["data"]) for row in qres]
items

[{'first': 'John',
  'last': 'Smith',
  'phone': {'home': '215.555.6587', 'work': '919.555.4795'},
  'kids': [{'first': 'John Jr.', 'last': 'Smith'},
   {'first': 'Jane', 'last': 'Smith'}]},
 {'first': 'Peggy',
  'last': 'Line',
  'phone': {'home': '505.555.3101'},
  'kids': [{'first': 'Jane', 'last': 'Line'},
   {'first': 'Molly', 'last': 'Line'}]}]

### Keys/Paths

We can also look into the items more. Recall that in these examples, there are lots of common keys but there doesn't have to be!

For this demo, let's first add some new items

In [13]:
db.insert({"new": "item", "and_with": {"multiple": "sub", "elements": None}})

Can see how often certains keys show

In [14]:
db.path_counts()

{'first': 4, 'kids': 4, 'last': 4, 'phone': 4, 'and_with': 1, 'new': 1}

Same for subkeys. These follow the standard options of JSON path string, regular single-item key, tuple, or query objects

Notice this is empty because there are no sub-elements to `'new'`

In [15]:
db.path_counts("new")

{}

But there are sub items for `and_with`

In [16]:
db.path_counts(db.Q.and_with)

{'elements': 1, 'multiple': 1}

#### Queries if paths exists

JSONLiteDB, and the underlying tools, treat items as if they contain every path with a default value of `None`. Normally this is fine since you can do something like:

In [17]:
list(db.query(db.Q.new != None))

[{'new': 'item', 'and_with': {'multiple': 'sub', 'elements': None}}]

But for the path `db.Q.and_with.elements`, the value *is* None

In [18]:
list(db.query(db.Q.and_with.elements != None))

[]

Instead use `query_by_path_exists()`. Note that `query_by_path_exists()` does not allow for multiple queries or combined conditions. Those can either be done manually with the results or in Python (or in SQL if you're really adventurous)

In [19]:
list(db.query_by_path_exists(db.Q.and_with.elements))

[{'new': 'item', 'and_with': {'multiple': 'sub', 'elements': None}}]

## Indexes, Unique Constraints, and Insertions

As mentioned in basic usage, an index can *dramatically* speed up queries. They can also be used to enfore uniqueness.

Consider a simple index:

In [20]:
db = JSONLiteDB(":memory:")
db.insert({"make": "Ford", "model": "Mustang", "color": "red", "orders": 15})
db.insert({"make": "Honda", "model": "Accord", "color": "blue", "orders": 35})
db.insert({"make": "Kia", "model": "Telluride", "color": "red", "orders": 8})

db.create_index("color")
db.indexes

{'ix_items_9852c203': ['$."color"']}

Now we can query by color quickly. What about by make? Use a "unique" index

In [21]:
db.drop_index_by_name("ix_items_9852c203")

In [22]:
db.create_index("make", unique=True)
db.indexes

{'ix_items_e97e5129_UNIQUE': ['$."make"']}

In [23]:
try:
    db.insert({"make": "Ford", "model": "F-150", "color": "green"})
except sqlite3.IntegrityError as E:
    print(f"Raised {type(E).__name__}: {E}")

Raised IntegrityError: UNIQUE constraint failed: index 'ix_items_e97e5129_UNIQUE'


You can ignore it:

In [24]:
db.insert({"make": "Ford", "model": "F-150", "color": "green"}, duplicates="ignore")
list(db.query(make="Ford"))  # Unchanged

[{'make': 'Ford', 'model': 'Mustang', 'color': 'red', 'orders': 15}]

or replace it

In [25]:
db.insert({"make": "Ford", "model": "F-150", "color": "green"}, duplicates="replace")
list(db.query(make="Ford"))  # Now it's the F-150

[{'make': 'Ford', 'model': 'F-150', 'color': 'green'}]

You can, of coure, delete the index

In [26]:
db.drop_index("make")  # or db.drop_index_by_name('ix_items_e97e5129')

Note that an index can be on multiple items and/or subqueries. Basically, anything that can be queried, can be made into an index. Do note SQLite is *extremely* particular about the index. An index on `'make'` maps to an index on `'$."make"'` which is functionally the same, but won't be used for a query on `'$.make'`. See below.

In [27]:
db.create_index("make")  # Same as db.create_index(db.Q.make)
db.create_index("$.make")  # Functionally the same but different index
db.indexes

{'ix_items_e97e5129_UNIQUE': ['$."make"'],
 'ix_items_e97e5129': ['$."make"'],
 'ix_items_1d9d7315': ['$.make']}

### Updating Rows

First, note that while the database returns mutable objects (a dict with the `rowid` attribute), it will not save those unless updated!

If those objects are updated, they can be passed back. The `rowid` must either be specified (precendance) or infered from the attribute

In [28]:
row = db.query_one(make="Ford")
row["orders"] = 900
db.update(row)

db.query_one(make="Ford")  # Will have 900 orders

{'make': 'Ford', 'model': 'F-150', 'color': 'green', 'orders': 900}

## Aggregate Functions

Data can also be aggregated with AVG COUNT MAX MIN SUM TOTAL either directly or with a method of that name. See https://www.sqlite.org/lang_aggfunc.html for description.

Using the same databse

In [29]:
db.AVG("orders")

314.3333333333333

In [30]:
db.MIN("orders")

8

In [31]:
db.MAX("orders")

900

In [32]:
db.aggregate("orders", "SUM")  # == db.SUM('orders')

943

## Deletions

In [33]:
db.delete({"make": "Ford"})

Can also do with a context manager to handle multiple in the same transaction

In [34]:
with db:
    db.delete({"make": "Honda"})
    db.delete({"make": "Nissan"})

And can also delete one or more by rowid if known.

In [35]:
rowid = db.query(db.Q.make == "Kia").one().rowid
print(f"{rowid = }")
db.delete_by_rowid(rowid)

rowid = 3


## Recipes

You can build other useful objects out of JSONLiteDB. Either with correct methods to match a Python object or just use it however. Note that there may be ready-made tools that do something similar and that is fine. These just let you use the simple nature of JSONLiteDB.

These examples all use `":memory:"` but the idea is that you can then persist them to a file


### Key-Value Store

Simple as a `{'key':'mykey','value':'myvalue'}` items with a unique index

In [36]:
kv = JSONLiteDB(":memory:")
kv.create_index("key", unique=True)

kv.insert({"key": "mykey", "value": "myvalue"})
# ...

### List

Just make object with `None` value

In [37]:
mylist = JSONLiteDB(":memory:")
mylist.insert("item1", "item2", "item1")

# mylist.create_index(tuple()) # OPTIONAL

In [38]:
"item1" in mylist

True

In [39]:
"item4" in mylist

False

### Set

Just like a list but create an empty unique index

In [40]:
myset = JSONLiteDB(":memory:")
myset.insert("item1", "item2")

myset.create_index(tuple(), unique=True)  # Required

In [41]:
"item1" in myset

True

In [42]:
"item3" in myset

False

In [43]:
# myset.add('item1')
# IntegrityError: UNIQUE constraint failed: index 'ix_items_c3e97dd6_UNIQUE'

## Side Note: Query Objects

Just to note that the following are all the same for a given `db`:

In [44]:
db.Q.key == "val"  # No call
db.Q().key == "val"  # Called. Optional

from jsonlitedb import Query, Q

Query().key == "val"  # MUST be called
Q().key == "val"  # MUST be called

# See them:
print(f'{db.Q.key == "val" = }')
print(f'{db.Q.key.subkey[3] @ "val" = }')

db.Q.key == "val" = Query(( JSON_EXTRACT(data, '$."key"') = 'val' ))
db.Q.key.subkey[3] @ "val" = Query(( JSON_EXTRACT(data, '$."key"."subkey"[3]') REGEXP 'val' ))
